In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import PolynomialFeatures

In [2]:
data = pd.read_csv('smoking_driking_dataset_Ver01.csv')

In [3]:
data.head()

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
0,Male,35,170,75,90.0,1.0,1.0,1.0,1.0,120.0,...,126.0,92.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,Y
1,Male,30,180,80,89.0,0.9,1.2,1.0,1.0,130.0,...,148.0,121.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,N
2,Male,40,165,75,91.0,1.2,1.5,1.0,1.0,120.0,...,74.0,104.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,N
3,Male,50,175,80,91.0,1.5,1.2,1.0,1.0,145.0,...,104.0,106.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,N
4,Male,50,165,60,80.0,1.0,1.2,1.0,1.0,138.0,...,117.0,104.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,N


### Understanding the size of the dataset

In [4]:
data.shape

(991346, 24)

### Understanding the uniqueness of the data

In [5]:
data.nunique()

sex                    2
age                   14
height                13
weight                24
waistline            737
sight_left            24
sight_right           24
hear_left              2
hear_right             2
SBP                  171
DBP                  127
BLDS                 498
tot_chole            474
HDL_chole            223
LDL_chole            432
triglyceride        1657
hemoglobin           190
urine_protein          6
serum_creatinine     183
SGOT_AST             568
SGOT_ALT             594
gamma_GTP            940
SMK_stat_type_cd       3
DRK_YN                 2
dtype: int64

### Finding missing values 

In [6]:
data.isnull().sum()

sex                 0
age                 0
height              0
weight              0
waistline           0
sight_left          0
sight_right         0
hear_left           0
hear_right          0
SBP                 0
DBP                 0
BLDS                0
tot_chole           0
HDL_chole           0
LDL_chole           0
triglyceride        0
hemoglobin          0
urine_protein       0
serum_creatinine    0
SGOT_AST            0
SGOT_ALT            0
gamma_GTP           0
SMK_stat_type_cd    0
DRK_YN              0
dtype: int64

In [7]:
data[data.duplicated].shape

(26, 24)

### NVRN nado udalit' duplicate

In [8]:
data = data.drop_duplicates(keep='first')
data[data.duplicated].shape

(0, 24)

### Description of dataset

In [9]:
data.describe()

,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,DBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
count,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,...,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000,991320.000000
mean,47.614529,162.240563,63.283884,81.233255,0.980833,0.978428,1.031495,1.030476,122.432360,76.052549,...,56.936984,113.037429,132.140030,14.229810,1.094221,0.860467,25.989424,25.755148,37.136152,1.608112
std,14.181346,9.282922,12.514101,11.850296,0.605954,0.604779,0.174652,0.171892,14.543083,9.889334,...,17.238578,35.842938,102.194762,1.584924,0.437719,0.480536,23.493668,26.308910,50.423811,0.818504
min,20.000000,130.000000,25.000000,8.000000,0.100000,0.100000,1.000000,1.000000,67.000000,32.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.100000,1.000000,1.000000,1.000000,1.000000
25%,35.000000,155.000000,55.000000,74.100000,0.700000,0.700000,1.000000,1.000000,112.000000,70.000000,...,46.000000,89.000000,73.000000,13.200000,1.000000,0.700000,19.000000,15.000000,16.000000,1.000000
50%,45.000000,160.000000,60.000000,81.000000,1.000000,1.000000,1.000000,1.000000,120.000000,76.000000,...,55.000000,111.000000,106.000000,14.300000,1.000000,0.800000,23.000000,20.000000,23.000000,1.000000
75%,60.000000,170.000000,70.000000,87.800000,1.200000,1.200000,1.000000,1.000000,131.000000,82.000000,...,66.000000,135.000000,159.000000,15.400000,1.000000,1.000000,28.000000,29.000000,39.000000,2.000000
max,85.000000,190.000000,140.000000,999.000000,9.900000,9.900000,2.000000,2.000000,273.000000,185.000000,...,8110.000000,5119.000000,9490.000000,25.000000,6.000000,98.000000,9999.000000,7210.000000,999.000000,3.000000


## Halving the dataset

In [10]:
sample_fraction = 0.5  # 50% of the data
data = data.sample(frac=sample_fraction, random_state=42)

In [11]:
data.describe()

,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,DBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
count,495660.000000,495660.000000,495660.000000,495660.000000,495660.000000,495660.000000,495660.000000,495660.00000,495660.000000,495660.000000,...,495660.000000,495660.000000,495660.000000,495660.000000,495660.000000,495660.000000,495660.000000,495660.000000,495660.000000,495660.000000
mean,47.623946,162.238853,63.289624,81.246180,0.981431,0.978003,1.031017,1.03038,122.458316,76.069382,...,56.905336,113.085801,132.137631,14.232272,1.094664,0.860748,25.971000,25.758885,37.203736,1.608657
std,14.189061,9.282599,12.518625,12.100261,0.608097,0.600509,0.173364,0.17163,14.547938,9.903695,...,15.318974,35.742586,101.207164,1.585043,0.439104,0.514933,20.628256,25.782976,50.887814,0.818407
min,20.000000,130.000000,25.000000,30.000000,0.100000,0.100000,1.000000,1.00000,72.000000,34.000000,...,1.000000,1.000000,5.000000,1.000000,1.000000,0.100000,1.000000,1.000000,1.000000,1.000000
25%,35.000000,155.000000,55.000000,74.200000,0.700000,0.700000,1.000000,1.00000,112.000000,70.000000,...,46.000000,89.000000,73.000000,13.200000,1.000000,0.700000,19.000000,15.000000,16.000000,1.000000
50%,45.000000,160.000000,60.000000,81.000000,1.000000,1.000000,1.000000,1.00000,120.000000,76.000000,...,55.000000,111.000000,106.000000,14.300000,1.000000,0.800000,23.000000,20.000000,23.000000,1.000000
75%,60.000000,170.000000,70.000000,87.800000,1.200000,1.200000,1.000000,1.00000,131.000000,82.000000,...,66.000000,135.000000,159.000000,15.400000,1.000000,1.000000,28.000000,29.000000,39.000000,2.000000
max,85.000000,190.000000,140.000000,999.000000,9.900000,9.900000,2.000000,2.00000,255.000000,185.000000,...,1206.000000,2254.000000,5236.000000,25.000000,6.000000,98.000000,7000.000000,4633.000000,999.000000,3.000000


# Preprocessing of the dataset

### Converting categorical values

In [12]:
data['sex'] = data['sex'].map({'Male': 1, 'Female': 0})
data['DRK_YN'] = data['DRK_YN'].map({'Y': 1, 'N': 0})


X = data.drop(['DRK_YN', 'SMK_stat_type_cd'], axis=1)
y = data['DRK_YN']

display(data.head())

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
857155,1,45,165,70,91.0,0.9,1.5,1.0,1.0,138.0,...,69.0,142.0,15.8,1.0,0.8,28.0,20.0,60.0,3.0,1
248823,1,75,165,60,85.0,0.6,0.7,1.0,1.0,165.0,...,43.0,205.0,14.9,1.0,0.8,13.0,16.0,23.0,1.0,0
903710,1,35,165,65,76.0,1.2,1.2,1.0,1.0,134.0,...,138.0,107.0,16.6,1.0,0.9,17.0,19.0,33.0,3.0,1
59866,0,60,145,50,80.0,0.6,0.9,1.0,1.0,116.0,...,69.0,251.0,12.2,5.0,1.3,21.0,12.0,13.0,1.0,0
192679,0,55,155,45,60.0,1.5,1.0,1.0,1.0,100.0,...,110.0,101.0,13.2,1.0,0.9,16.0,12.0,14.0,1.0,0


In [13]:
# Verification
# Check if the specified columns are dropped from X
print("Columns in X after dropping: ", X.columns.tolist())
print("First few rows of X:\n", X.head())

# Check if y contains the correct target variable
print("First few rows of y:\n", y.head())

print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)

Columns in X after dropping:  ['sex', 'age', 'height', 'weight', 'waistline', 'sight_left', 'sight_right', 'hear_left', 'hear_right', 'SBP', 'DBP', 'BLDS', 'tot_chole', 'HDL_chole', 'LDL_chole', 'triglyceride', 'hemoglobin', 'urine_protein', 'serum_creatinine', 'SGOT_AST', 'SGOT_ALT', 'gamma_GTP']
First few rows of X:
         sex  age  height  weight  waistline  sight_left  sight_right  \
857155    1   45     165      70       91.0         0.9          1.5   
248823    1   75     165      60       85.0         0.6          0.7   
903710    1   35     165      65       76.0         1.2          1.2   
59866     0   60     145      50       80.0         0.6          0.9   
192679    0   55     155      45       60.0         1.5          1.0   

        hear_left  hear_right    SBP  ...  tot_chole  HDL_chole  LDL_chole  \
857155        1.0         1.0  138.0  ...      149.0       52.0       69.0   
248823        1.0         1.0  165.0  ...      118.0       34.0       43.0   
903710      

In [14]:
for feature in data:
    print(feature)

sex
age
height
weight
waistline
sight_left
sight_right
hear_left
hear_right
SBP
DBP
BLDS
tot_chole
HDL_chole
LDL_chole
triglyceride
hemoglobin
urine_protein
serum_creatinine
SGOT_AST
SGOT_ALT
gamma_GTP
SMK_stat_type_cd
DRK_YN


In [15]:
# Checking if there are outliers
for feature in data:
    
    Q1 = data[feature].quantile(0.25)
    Q3 = data[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    
    if data[(data[feature] > upper)].any(axis=None):
        print(feature,"yes")
    else:
        print(feature, "no")

sex no
age no
height no
weight yes
waistline yes
sight_left yes
sight_right yes
hear_left yes
hear_right yes
SBP yes
DBP yes
BLDS yes
tot_chole yes
HDL_chole yes
LDL_chole yes
triglyceride yes
hemoglobin yes
urine_protein yes
serum_creatinine yes
SGOT_AST yes
SGOT_ALT yes
gamma_GTP yes
SMK_stat_type_cd no
DRK_YN no


In [16]:
outliers_list = []

for feature in data.columns:
    Q1 = data[feature].quantile(0.05)
    Q3 = data[feature].quantile(0.95)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = data[(data[feature] < lower) | (data[feature] > upper)]
    
    if not outliers.empty:
        outliers = outliers.copy()  # Create a copy to avoid SettingWithCopyWarning
        outliers['outlier_feature'] = feature  # Add column to indicate outlier feature
        outliers_list.append(outliers)
        print(f"{feature}: yes")
    else:
        print(f"{feature}: no")

sex: no
age: no
height: no
weight: no
waistline: yes
sight_left: yes
sight_right: yes
hear_left: yes
hear_right: yes
SBP: yes
DBP: yes
BLDS: yes
tot_chole: yes
HDL_chole: yes
LDL_chole: yes
triglyceride: yes
hemoglobin: yes
urine_protein: yes
serum_creatinine: yes
SGOT_AST: yes
SGOT_ALT: yes
gamma_GTP: yes
SMK_stat_type_cd: no
DRK_YN: no


In [17]:
if outliers_list:
    all_outliers = pd.concat(outliers_list)
    all_outliers.drop_duplicates()
    outlier_count = all_outliers.shape[0]
    
    print(f"\nNumber of extreme outliers in the dataset: {outlier_count}")
    print("\nSome rows with extreme outliers:")
    print(all_outliers.head())
    
else:
    print("\nNo outliers found in the dataset.")


Number of extreme outliers in the dataset: 54967

Some rows with extreme outliers:
        sex  age  height  weight  waistline  sight_left  sight_right  \
125594    0   30     155      60      999.0         0.9          0.7   
278576    0   30     160      65      999.0         1.2          1.2   
194638    0   35     160      60      999.0         0.4          0.5   
252103    0   30     165      60      999.0         1.2          0.9   
754482    0   25     150      60      999.0         1.2          1.2   

        hear_left  hear_right    SBP  ...  triglyceride  hemoglobin  \
125594        1.0         1.0  107.0  ...          86.0        11.2   
278576        1.0         1.0  110.0  ...         186.0        12.5   
194638        1.0         1.0  113.0  ...         248.0        10.8   
252103        1.0         1.0  110.0  ...         240.0        13.5   
754482        1.0         1.0  109.0  ...         120.0        13.7   

        urine_protein  serum_creatinine  SGOT_AST  SGOT_

In [18]:
data = data.drop(all_outliers.index)
data = data.drop_duplicates()
print("\nDataFrame after removing extreme outliers:")
print(data.head())


DataFrame after removing extreme outliers:
        sex  age  height  weight  waistline  sight_left  sight_right  \
857155    1   45     165      70       91.0         0.9          1.5   
248823    1   75     165      60       85.0         0.6          0.7   
903710    1   35     165      65       76.0         1.2          1.2   
192679    0   55     155      45       60.0         1.5          1.0   
653978    1   35     175      75       83.0         1.2          1.5   

        hear_left  hear_right    SBP  ...  LDL_chole  triglyceride  \
857155        1.0         1.0  138.0  ...       69.0         142.0   
248823        1.0         1.0  165.0  ...       43.0         205.0   
903710        1.0         1.0  134.0  ...      138.0         107.0   
192679        1.0         1.0  100.0  ...      110.0         101.0   
653978        1.0         1.0  110.0  ...      142.0          62.0   

        hemoglobin  urine_protein  serum_creatinine  SGOT_AST  SGOT_ALT  \
857155        15.8         

In [19]:
data.describe()

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
count,454746.000000,454746.000000,454746.000000,454746.000000,454746.000000,454746.000000,454746.000000,454746.0,454746.0,454746.000000,...,454746.000000,454746.000000,454746.000000,454746.000000,454746.000000,454746.000000,454746.000000,454746.000000,454746.000000,454746.000000
mean,0.523895,46.779477,162.366563,63.236664,80.889042,0.963842,0.961306,1.0,1.0,121.990115,...,113.350486,126.717704,14.228445,1.061296,0.848135,24.682040,24.316097,33.030778,1.600863,0.503094
std,0.499429,13.857025,9.233849,12.456478,9.559764,0.339224,0.337835,0.0,0.0,14.343826,...,34.112382,80.783328,1.573024,0.296914,0.197722,9.458532,15.589608,30.322405,0.816778,0.499991
min,0.000000,20.000000,130.000000,25.000000,30.000000,0.100000,0.100000,1.0,1.0,72.000000,...,1.000000,5.000000,4.400000,1.000000,0.100000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,35.000000,155.000000,55.000000,74.000000,0.700000,0.700000,1.0,1.0,111.000000,...,90.000000,72.000000,13.200000,1.000000,0.700000,19.000000,14.000000,15.000000,1.000000,0.000000
50%,1.000000,45.000000,160.000000,60.000000,81.000000,1.000000,1.000000,1.0,1.0,120.000000,...,111.000000,105.000000,14.300000,1.000000,0.800000,23.000000,20.000000,23.000000,1.000000,1.000000
75%,1.000000,55.000000,170.000000,70.000000,87.000000,1.200000,1.200000,1.0,1.0,130.000000,...,135.000000,156.000000,15.400000,1.000000,1.000000,28.000000,29.000000,38.000000,2.000000,1.000000
max,1.000000,85.000000,190.000000,140.000000,138.000000,2.500000,2.500000,1.0,1.0,220.000000,...,336.000000,673.000000,23.600000,3.000000,2.000000,92.000000,132.000000,247.000000,3.000000,1.000000


In [20]:
data.to_csv('preprocessed_dataset.csv', index=False)

# Feature engineering

In [22]:
# Train a Random Forest model
#model_rf_feng = RandomForestClassifier(n_estimators=100, random_state=0)
#model_rf_feng.fit(X, y)

#joblib.dump(model_rf_feng, 'random_forest_model_feng.pkl')
model_rf_feng = joblib.load('random_forest_model_feng.pkl')

# Get feature importances
importances = model_rf_feng.feature_importances_

# Create a DataFrame for visualization
importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Select top N features
N = 10
top_features = importances_df['Feature'].head(N).tolist()

print("Top features based on tree-based importance:", top_features)



Top features based on tree-based importance: ['gamma_GTP', 'hemoglobin', 'age', 'HDL_chole', 'sex', 'height', 'triglyceride', 'LDL_chole', 'tot_chole', 'SGOT_ALT']


In [23]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [24]:
#!pip install shap

DEPRECATION: Loading egg at c:\users\temir\anaconda3\lib\site-packages\skfeature-1.0.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


# Не работает frac

In [ ]:

import shap

data_shap = data.sample(frac=0.1, random_state=42)

# Train the model
model_rf_shap = RandomForestClassifier(random_state=42)
model_rf_shap.fit(X_train, y_train)

joblib.dump(model_rf_shap, 'random_forest_model_shap.pkl')
model_rf_shap = joblib.load('random_forest_model_shap.pkl')

# Initialize SHAP explainer
explainer = shap.TreeExplainer(model_rf_shap)
shap_values = explainer.shap_values(X_train)

# Visualize the feature importance
shap.summary_plot(shap_values[1], X_train, feature_names=feature_names)

In [ ]:
# Initialize the RobustScaler
scaler = RobustScaler()

# Fit the scaler on the training data and transform both the training and testing data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Initial Classification Quality Measures:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))

In [ ]:
# Display the first few rows of the scaled training data
print("Scaled Training Data:")
display(pd.DataFrame(X_train).head())

# Display the first few rows of the scaled testing data
print("Scaled Testing Data:")
display(pd.DataFrame(X_test).head())


In [ ]:
# Initialize models
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "KNN": KNeighborsClassifier()
}

# Train and evaluate models
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Save the trained model
    joblib.dump(model, f'{model_name}_model.pkl')
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)
    
    # Print the evaluation metrics
    print(f'--- {model_name} Model ---')
    print(f'Accuracy: {accuracy}')
    print('Classification Report:')
    print(classification_rep)
    print('\n')

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

# Initialize the SVM model
svm = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(svm, param_grid, refit=True, verbose=3)

# Fit GridSearchCV on the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print(f'Best parameters found: {grid_search.best_params_}')


# Get the best SVM model from GridSearchCV
best_svm = grid_search.best_estimator_

# Save the best SVM model
joblib.dump(best_svm, 'best_svm_model.pkl')

# Predict on the test set using the best SVM model
y_pred = best_svm.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f'--- SVM Model ---')
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_rep)

# Print confusion matrix to further diagnose
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
